In [7]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pickle
import sys
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/genome3D/')
from models_3d import Genome3D_RandomForest
from metrics import ClassificationResult
from utils import normalize_features_sampl_by_f, concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
contacts='pe'
path='/users/mtaranov/datasets_3d/by_chr_dist_matched_'+contacts+'/'
day='d0'
thres='10'

X_train_node1 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_train_node2 = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_train_window = path+'motifs/'+day+'_thres_'+thres+'_train_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'


X_test_node1 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_test_node2 = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_test_window = path+'motifs/'+day+'_thres_'+thres+'_test_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

X_valid_node1 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_valid_node2 = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_valid_window = path+'motifs/'+day+'_thres_'+thres+'_valid_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy'

y_train = get_labels(path+day+'_y_train_thres_'+thres+'.npy')
y_test = get_labels(path+day+'_y_test_thres_'+thres+'.npy')
y_valid = get_labels(path+day+'_y_valid_thres_'+thres+'.npy')

X_train=concat_motifs([X_train_node1, X_train_node2, X_train_window])
X_test=concat_motifs([X_test_node1, X_test_node2, X_test_window])
X_valid=concat_motifs([X_valid_node1, X_valid_node2, X_valid_window])

In [4]:
X_train_pairs = X_train.reshape(X_train.shape[0],X_train.shape[2]*X_train.shape[3])
X_valid_pairs = X_valid.reshape(X_valid.shape[0],X_valid.shape[2]*X_valid.shape[3])
X_test_pairs = X_test.reshape(X_test.shape[0],X_test.shape[2]*X_test.shape[3])

In [5]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features_sampl_by_f(X_train_pairs, X_valid_pairs, X_test_pairs)

# Random Forest

### with motifs only

# Estimating hyper-parameter using held out validation set

In [6]:
# X_train_valid = np.concatenate((X_train_normalized, X_valid_normalized), axis=0)
# y_train_valid = np.concatenate((y_train, y_valid), axis=0)
# test_fold to 0 for all samples that are part of the validation set, and to -1 for all other samples.
# valid_index=[-1 for i in range(X_train_normalized.shape[0])]+[0 for i in range(X_valid_normalized.shape[0])]
# param_grid = {'n_estimators': [100, 200, 500], 'criterion': ['gini', 'entropy'], 'max_features': ['auto', 'log2' ], 'max_depth': [None, 2, 3, 5], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [1, 2, 3], 'min_impurity_split': [0.0 , 0.1, 0.2]}
# best_param={}
# rf = Genome3D_RandomForest(best_param)
# best_param = rf.train_cross_val(X_train_valid[:,:], [i for i in y_train_valid[:,0]], valid_index, param_grid)

# Random Forest Predictions

In [8]:
best_param = pickle.load(open( '/users/mtaranov/genome3D/examples/from_sequence_features/nodes_and_edge_features/best_param_rf.pkl', 'rb' ) )

In [9]:
rf = Genome3D_RandomForest(best_param)

In [10]:
rf.train(X_train_normalized, y_train)
preds_test = rf.predict(X_test_normalized)
preds_train = rf.predict(X_train_normalized)
print ('On Test:\n{}\n'.format(ClassificationResult(y_test, preds_test)))
print ('On Train:\n{}\n'.format(ClassificationResult(y_train, preds_train)))

/users/mtaranov/genome3D/models_3d.py:269: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.classifier.fit(X, y)


On Test:
Balanced Accuracy: 64.27%	 auROC: 0.667	 auPRC: 0.638	 auPRG: 0.370
Positve Accuracy: 76.32%	 Negative Accuracy: 52.23%
Recall at 5%|10%|20% FDR: 2.4%|2.8%|7.3%	 Num Positives: 494	 Num Negatives: 494	 

On Train:
Balanced Accuracy: 100.00%	 auROC: 1.000	 auPRC: 1.000	 auPRG: 1.000
Positve Accuracy: 100.00%	 Negative Accuracy: 100.00%
Recall at 5%|10%|20% FDR: 100.0%|100.0%|100.0%	 Num Positives: 3355	 Num Negatives: 3355	 



# Random Forest Feature Selection

In [11]:
scores=rf.ImportanceSelect()

In [12]:
scores.shape

(5760,)

In [13]:
motif_names=np.loadtxt('/users/mtaranov/datasets_3d/dist_matched_PE/motifs/train_PE_out_btw_nodes_3_0.0001/motif_names.txt', dtype=str, delimiter='/')[:,6:].reshape(640,)
top_3_hits_names=[]
for i in motif_names:
    for j in range(3):
        top_3_hits_names.append(i)
top_3_hits_names = np.array(top_3_hits_names)

In [14]:
score_per_feature_node1=scores[0:5760:3]
score_per_feature_node2=scores[1:5761:3]
score_per_feature_window=scores[2:5762:3]

In [15]:
RF_Importance_Motifs_node1=top_3_hits_names[score_per_feature_node1.argsort()[::-1]]
RF_Importance_Motifs_node2=top_3_hits_names[score_per_feature_node2.argsort()[::-1]]
RF_Importance_Motifs_window=top_3_hits_names[score_per_feature_window.argsort()[::-1]]

In [16]:
print np.where(RF_Importance_Motifs_node1=='CTCF_HUMAN')
print np.where(RF_Importance_Motifs_node1=='CTCFL_HUMAN')

(array([ 384, 1138, 1394]),)
(array([233, 400, 741]),)


In [17]:
print np.where(RF_Importance_Motifs_node2=='CTCF_HUMAN')
print np.where(RF_Importance_Motifs_node2=='CTCFL_HUMAN')

(array([ 239,  252, 1017]),)
(array([ 112,  187, 1105]),)


In [18]:
print np.where(RF_Importance_Motifs_window=='CTCF_HUMAN')
print np.where(RF_Importance_Motifs_window=='CTCFL_HUMAN')

(array([ 460,  917, 1366]),)
(array([161, 168, 467]),)


In [19]:
RF_Importance_Motifs_node1[:100]

array(['HXB6_HUMAN', 'ELF3_HUMAN', 'SOX15_HUMAN', 'ARI5B_HUMAN',
       'HMX3_HUMAN', 'RFX2_HUMAN', 'SMAD2_HUMAN', 'ARI5B_HUMAN',
       'P5F1B_HUMAN', 'FOXJ2_HUMAN', 'TBX15_HUMAN', 'SOX15_HUMAN',
       'ISX_HUMAN', 'VSX2_HUMAN', 'TBX15_HUMAN', 'RFX2_HUMAN',
       'VAX2_HUMAN', 'SP4_HUMAN', 'HMBX1_HUMAN', 'IRF5_HUMAN',
       'P5F1B_HUMAN', 'ARI5B_HUMAN', 'HXA1_HUMAN', 'ZN282_HUMAN',
       'SMAD2_HUMAN', 'NFAC3_HUMAN', 'NFIA_HUMAN', 'IRX2_HUMAN',
       'HMX3_HUMAN', 'MEOX2_HUMAN', 'TBP_HUMAN', 'SP4_HUMAN', 'SOX5_HUMAN',
       'TAL1_HUMAN', 'IKZF1_HUMAN', 'NKX28_HUMAN', 'DLX4_HUMAN',
       'MEIS2_HUMAN', 'GCR_HUMAN', 'HXA1_HUMAN', 'HLTF_HUMAN',
       'IRF5_HUMAN', 'PLAG1_HUMAN', 'HXC10_HUMAN', 'TBX20_HUMAN',
       'HXA1_HUMAN', 'NKX28_HUMAN', 'SOX21_HUMAN', 'GATA1_HUMAN',
       'HIC1_HUMAN', 'DLX4_HUMAN', 'ETV3_HUMAN', 'IRF5_HUMAN', 'SP1_HUMAN',
       'RFX1_HUMAN', 'HXB7_HUMAN', 'FOXD2_HUMAN', 'TAL1_HUMAN',
       'ZBTB4_HUMAN', 'IRF4_HUMAN', 'GCR_HUMAN', 'RARB_HUMAN',
       

In [20]:
RF_Importance_Motifs_node2[:100]

array(['MAFG_HUMAN', 'TBP_HUMAN', 'IRF5_HUMAN', 'EOMES_HUMAN', 'TBP_HUMAN',
       'TBX15_HUMAN', 'KLF4_HUMAN', 'TBX15_HUMAN', 'ZBED1_HUMAN',
       'KLF4_HUMAN', 'ZBED1_HUMAN', 'TAL1_HUMAN', 'ZBTB4_HUMAN',
       'P5F1B_HUMAN', 'MEIS2_HUMAN', 'SP1_HUMAN', 'GCR_HUMAN', 'TBP_HUMAN',
       'TBX4_HUMAN', 'MAFG_HUMAN', 'PO3F3_HUMAN', 'PRGR_HUMAN',
       'IRF4_HUMAN', 'ARI5B_HUMAN', 'LEF1_HUMAN', 'P5F1B_HUMAN',
       'ARI5B_HUMAN', 'HMBX1_HUMAN', 'SMAD2_HUMAN', 'RFX4_HUMAN',
       'IRF8_HUMAN', 'ZBTB4_HUMAN', 'ZN350_HUMAN', 'EOMES_HUMAN',
       'BACH1_HUMAN', 'TAL1_HUMAN', 'IRF4_HUMAN', 'ZN350_HUMAN',
       'IRF4_HUMAN', 'HNF1A_HUMAN', 'ZSC16_HUMAN', 'TAL1_HUMAN',
       'E2F4_HUMAN', 'HXA1_HUMAN', 'PAX5_HUMAN', 'MZF1_HUMAN',
       'HXA1_HUMAN', 'PO3F3_HUMAN', 'OLIG3_HUMAN', 'NKX28_HUMAN',
       'ARI5B_HUMAN', 'MAZ_HUMAN', 'NOBOX_HUMAN', 'TBX1_HUMAN',
       'PO5F1_HUMAN', 'ZBED1_HUMAN', 'NFAC3_HUMAN', 'ZBTB4_HUMAN',
       'SRBP2_HUMAN', 'MAFG_HUMAN', 'TBX1_HUMAN', 'TBX20_HUMAN',
 

In [21]:
RF_Importance_Motifs_window[:100]

array(['P63_HUMAN', 'HXB6_HUMAN', 'FOXD3_HUMAN', 'ELF3_HUMAN',
       'HXB6_HUMAN', 'LHX8_HUMAN', 'LHX2_HUMAN', 'E2F8_HUMAN',
       'ZBT18_HUMAN', 'CRX_HUMAN', 'FOXD3_HUMAN', 'NFYA_HUMAN',
       'HXA7_HUMAN', 'BARX2_HUMAN', 'GLI1_HUMAN', 'GLI1_HUMAN',
       'ALX4_HUMAN', 'HXB6_HUMAN', 'P63_HUMAN', 'SRY_HUMAN', 'PKNX2_HUMAN',
       'GRHL1_HUMAN', 'HXA7_HUMAN', 'ELK1_HUMAN', 'ETV3_HUMAN',
       'OLIG3_HUMAN', 'ETV3_HUMAN', 'ZN423_HUMAN', 'EMX1_HUMAN',
       'FLI1_HUMAN', 'CREB3_HUMAN', 'ALX4_HUMAN', 'DUXA_HUMAN',
       'LHX8_HUMAN', 'SOX13_HUMAN', 'SPIC_HUMAN', 'HXD8_HUMAN',
       'ELK1_HUMAN', 'ZN589_HUMAN', 'PKNX2_HUMAN', 'SRY_HUMAN',
       'ELF3_HUMAN', 'PAX3_HUMAN', 'CREB3_HUMAN', 'ELF3_HUMAN',
       'HXD12_HUMAN', 'ERR1_HUMAN', 'MAFA_HUMAN', 'FOXC1_HUMAN',
       'RARG_HUMAN', 'EMX1_HUMAN', 'HXC8_HUMAN', 'ARI5B_HUMAN',
       'NFAC1_HUMAN', 'DUXA_HUMAN', 'HXC8_HUMAN', 'FOXK1_HUMAN',
       'RARG_HUMAN', 'TFDP1_HUMAN', 'HMGA2_HUMAN', 'THAP1_HUMAN',
       'TEAD1_HUMAN', 'FL

In [22]:
from sets import Set
uniq_RF_Importance_Motifs_node1 = Set(RF_Importance_Motifs_node1[:100])
uniq_RF_Importance_Motifs_node2 = Set(RF_Importance_Motifs_node2[:100])
uniq_RF_Importance_Motifs_window = Set(RF_Importance_Motifs_window[:100])


/users/mtaranov/local/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: the sets module is deprecated
  if __name__ == '__main__':


In [23]:
uniq_RF_Importance_Motifs_node1 & uniq_RF_Importance_Motifs_node2 & uniq_RF_Importance_Motifs_window


Set(['PKNX2_HUMAN', 'ARI5B_HUMAN', 'ETV3_HUMAN'])